In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)

# Inicia el navegador con la configuración headless
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Ruta del archivo Excel donde se almacenarán los productos
ruta_excel = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\FA1.xlsx"

# Cargar el archivo Excel existente (si existe)
try:
    df_existente = pd.read_excel(ruta_excel)
    print("Archivo Excel cargado con éxito.")
except FileNotFoundError:
    df_existente = pd.DataFrame(columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])
    print("No se encontró el archivo Excel. Se creará uno nuevo.")

# Lista para almacenar los productos extraídos
productos_extraidos = []

# Función para extraer la información de los productos en una página
def extraer_productos(url):
    driver.get(url)
    
    # Esperar hasta que los resultados de búsqueda estén completamente cargados
    try:
        # Esperar hasta que los resultados de productos estén visibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#testId-searchResults-products > div"))
        )
        
        # Buscar los productos
        productos = driver.find_elements(By.CSS_SELECTOR, "#testId-searchResults-products > div")
        
        if len(productos) == 0:
            print("No se encontraron productos en esta página. Finalizando el scraping.")
            return False
        
        # Iterar sobre los productos y extraer los datos
        for producto in productos:
            try:
                # Variable para almacenar la URL de la imagen
                url_imagen = ""

                # Extraer la URL de la imagen (si está disponible)
                try:
                    img_url = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
                    if img_url and img_url.endswith("fit=pad"):
                        url_imagen = img_url
                except:
                    url_imagen = ""
                
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "b.pod-subTitle.subTitle-rebrand").text.strip()

                # Extraer el precio original (si existe)
                try:
                    precio_original = producto.find_element(By.CSS_SELECTOR, "li[data-normal-price]").get_attribute('data-normal-price')
                except:
                    precio_original = ""

                # Extraer el precio final (si existe)
                try:
                    precio_final = producto.find_element(By.CSS_SELECTOR, "li[data-internet-price]").get_attribute('data-internet-price')
                except:
                    precio_final = ""

                # Extraer la URL del producto
                url_producto = producto.find_element(By.CSS_SELECTOR, "a").get_attribute('href')

                # Extraer el SKU
                sku = ""  # Asignar valor predeterminado

                # Extraer la marca, que está en el texto de la etiqueta <b> con id 'testId-pod-displaySellerText'
                try:
                    marca_texto = producto.find_element(By.CSS_SELECTOR, "b[id^='testId-pod-displaySellerText']").text
                    if marca_texto.startswith("Por "):
                        marca = marca_texto[3:].strip()  # Eliminar el "Por " y obtener el nombre de la marca
                    else:
                        marca = ""  # Si no tiene "Por", no extraemos nada
                except:
                    marca = ""  # Si no se encuentra la marca, asigna un valor vacío
                
                # Guardar los datos en la lista
                productos_extraidos.append({
                    "Nombre del Producto": nombre_producto,
                    "Precio Original": precio_original,
                    "Precio Final": precio_final,
                    "URL del Producto": url_producto,
                    "SKU": sku,
                    "URL de la Imagen": url_imagen,
                    "Marca": marca
                })

                # Imprimir los resultados
                print(f"Producto: {nombre_producto}")
                print(f"Precio original: {precio_original}")
                print(f"Precio final: {precio_final}")
                print(f"URL del producto: {url_producto}")
                print(f"URL de la imagen: {url_imagen}")
                print(f"Marca: {marca}")
                print("-" * 50)

            except Exception as e:
                print(f"Error al extraer datos del producto: {e}")
        
        return True  # Retorna True si se encontraron productos

    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Número de páginas a recorrer
pagina = 1
continuar = True

while continuar:
    print(f"Accediendo a la página {pagina}...")
    
    # URL de la página actual
    url_pagina = f"https://www.falabella.com/falabella-cl/category/cat5620004/Moda-Infantil?page={pagina}"
    
    # Extraer productos de la página actual
    continuar = extraer_productos(url_pagina)
    
    # Si se encontraron productos, pasar a la siguiente página
    if continuar:
        pagina += 1
        time.sleep(10)  # Espera de 10 segundos entre páginas
    else:
        print("No hay más productos o hubo un error. Terminando el scraping.")
        break

# Crear el DataFrame final con todos los productos
df_final = pd.DataFrame(productos_extraidos, columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Concatenar los productos extraídos con los productos existentes en el archivo Excel
df_actualizado = pd.concat([df_existente, df_final], ignore_index=True)

# Guardar el DataFrame final en el archivo Excel
df_actualizado.to_excel(ruta_excel, index=False)

# Cerrar el navegador
driver.quit()


No se encontró el archivo Excel. Se creará uno nuevo.
Accediendo a la página 1...
Producto: Chaqueta Cortaviento Forro Delgado Niña
Precio original: 
Precio final: 12.990
URL del producto: https://www.falabella.com/falabella-cl/product/883293946/Chaqueta-Cortaviento-Forro-Delgado-Nina-Yamp/883293951
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/883293951_1/width=240,height=240,quality=70,format=webp,fit=pad
Marca: Falabella
--------------------------------------------------
Producto: Pantalon Escolar Skinny Juvenil Gris
Precio original: 
Precio final: 24.990
URL del producto: https://www.falabella.com/falabella-cl/product/117206833/Pantalon-Escolar-Skinny-Juvenil-Gris-Maui-and-Sons/117206834?sponsoredClickData=%257B%2522isXLP%2522%253Atrue%252C%2522pageType%2522%253A%2522category%2522%252C%2522adType%2522%253A%2522products%2522%252C%2522platform%2522%253A%2522desktop%2522%252C%2522regionCode%2522%253A%2522cl%2522%252C%2522context%2


KeyboardInterrupt

